<a href="https://colab.research.google.com/github/AlexFlare/data-engineer-code-test/blob/main/pyspark_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load pretty big dataset from kaggle

! pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
! mkdir ~/.kaggle
! cp '/content/drive/MyDrive/ColabNotebooks/kaggle.json' ~/.kaggle/ # файл kaggle.json можно положить в любую папку в Drive, главное указать актуальный адрес в первом аргументе команды cp
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list
! kaggle datasets download -d 'karmukilandk/vibration-analysis-on-rotating-shaft'
! mkdir data
! unzip vibration-analysis-on-rotating-shaft.zip -d data

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/karmukilandk/vibration-analysis-on-rotating-shaft
License(s): MIT
 99% 2.38G/2.39G [00:50<00:00, 16.2MB/s]
100% 2.39G/2.39G [00:50<00:00, 50.6MB/s]
Archive:  vibration-analysis-on-rotating-shaft.zip
  inflating: data/cleaned_labeled_vibration_data.csv  


In [1]:
# Setup Spark

!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,772 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,574 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,1

In [3]:
df_spark = spark.read.csv('data/cleaned_labeled_vibration_data.csv', header=True, inferSchema=True)
df_spark.printSchema()

root
 |-- V_in: double (nullable = true)
 |-- Measured_RPM: double (nullable = true)
 |-- Vibration_1: double (nullable = true)
 |-- Vibration_2: double (nullable = true)
 |-- Vibration_3: double (nullable = true)
 |-- label: string (nullable = true)
 |-- unbalance_level: integer (nullable = true)
 |-- dataset_type: string (nullable = true)



In [4]:
df_spark.show()

+----+------------+-----------+-----------+-----------+-----+---------------+------------+
|V_in|Measured_RPM|Vibration_1|Vibration_2|Vibration_3|label|unbalance_level|dataset_type|
+----+------------+-----------+-----------+-----------+-----+---------------+------------+
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|
| 0.0|   28.610235|        0.0|        0.0|        0.0| Idle|              0|           D|

In [7]:
df_spark = df_spark.dropna()
#df_spark.pandas_api().isna().sum()